# Ettoday 網路爬蟲實作練習


* 能夠利用 Request + BeatifulSour 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例：

1. 取出今天所有的新聞
2. 取出現在時間兩小時內的新聞
3. 根據範例，取出三天前下午三點到五點的新聞

## 1. 取出今天所有的新聞

In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import re

In [2]:
today = time.strftime("%Y/%m/%d", time.localtime())
print(today)

2019/12/26


In [3]:
url = 'https://www.ettoday.net/news/news-list.htm'

driver = webdriver.Chrome(executable_path='chromedriver')
driver.get(url)


In [82]:
# 每2秒執行一次滾動視窗的JS程式碼
for i in range(10):
    time.sleep(2)
    driver.execute_script('window.scrollTo(1,10000);')

In [4]:
# 滾完了後，獲取html原始碼

html = driver.page_source
soup = BeautifulSoup(html, 'html5lib')
for h3 in soup.find(class_='part_list_2').find_all('h3'):
    if h3.find(class_='date').text.split(' ')[0] == today:
        Date = h3.find(class_='date').text
        Title = h3.find(name='a').text
        Category = h3.find(name='em').text
        Link = h3.find(name='a')['href']
        print(Date, Category, Title, 'https://www.ettoday.net'+Link)

2019/12/26 14:53 影劇 陳綺貞捲不倫戀...正宮喊告　前男友鍾成虎發聲力挺「心疼錯愕」 https://www.ettoday.net/news/20191226/1610914.htm
2019/12/26 14:51 政治 總統大選民調讓組頭崩潰！變身掮客避「倒莊」風險...藍綠對決先收賭金 https://www.ettoday.net/news/20191226/1610887.htm
2019/12/26 14:50 社會 警犯罪宣導轉空戰！　南港警邀藝人米可白開直播超吸睛 https://www.ettoday.net/news/20191226/1610901.htm
2019/12/26 14:48 社會 棄鴻海高薪轉戰調查局！台大電眼妹低調求學　爸「超硬背景」曝…同學全不知 https://www.ettoday.net/news/20191226/1610714.htm
2019/12/26 14:40 生活 2020年「新制懶人包」一次看！時薪150→158元、電動機車補助少了 https://www.ettoday.net/news/20191226/1610822.htm
2019/12/26 14:39 體育 聖誕大賽又爆冷！　英格雷31分領鵜鶘斬斷金塊7連勝 https://www.ettoday.net/news/20191226/1610910.htm
2019/12/26 14:39 消費 【廣編】樂天市場搶先公布2020消費新趨勢　STEM程式教育掀熱潮 https://www.ettoday.net/news/20191226/1610908.htm
2019/12/26 14:38 財經 200餘名台電工程師遭年末資遣　勞團要求「派遣歸零」 https://www.ettoday.net/news/20191226/1610906.htm
2019/12/26 14:34 財經 中租前11月獲利142億　每股盈餘破10元賺逾一個股本 https://www.ettoday.net/news/20191226/1610896.htm
2019/12/26 14:34 政治 告對手林宜瑾違反選罷法　林燕祝：絕不姑息新潮流的惡劣選戰手法 https://www.ettoday.net/news/20191226/1

## 2. 取出現在時間兩小時內的新聞

In [5]:
#把字串->時間->時戳 
def str_to_timestamp(strTime):
    string = str(strTime)
    timeArray = time.strptime(string, "%Y/%m/%d %H:%M")
    timestamp = time.mktime(timeArray)
    return timestamp

In [6]:
# 爬到的是字串－變成時戳－必須要大於 > 兩小時前的時戳 (小於就是比2小時前更早了)

two_hours_ago = float(time.time()) - 7200  # 兩小時前 = int(現在時戳秒數 - 7200秒)
Now = time.strftime('%Y/%m/%d %H:%M:%S',time.localtime())
Ago = time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(two_hours_ago))
print('現在時間:{}，獲取兩小時前:{}之後的新聞\n'.format(Now, Ago))

for h3 in soup.find(class_='part_list_2').find_all('h3'):
    date_string = h3.find(class_='date').text
    if str_to_timestamp(date_string) > two_hours_ago:
        Date = h3.find(class_='date').text
        Title = h3.find(name='a').text
        Category = h3.find(name='em').text
        Link = h3.find(name='a')['href']
        print(Date, Category, Title, 'https://www.ettoday.net'+Link)

現在時間:2019/12/26 14:55:33，獲取兩小時前:2019/12/26 12:55:33之後的新聞

2019/12/26 14:53 影劇 陳綺貞捲不倫戀...正宮喊告　前男友鍾成虎發聲力挺「心疼錯愕」 https://www.ettoday.net/news/20191226/1610914.htm
2019/12/26 14:51 政治 總統大選民調讓組頭崩潰！變身掮客避「倒莊」風險...藍綠對決先收賭金 https://www.ettoday.net/news/20191226/1610887.htm
2019/12/26 14:50 社會 警犯罪宣導轉空戰！　南港警邀藝人米可白開直播超吸睛 https://www.ettoday.net/news/20191226/1610901.htm
2019/12/26 14:48 社會 棄鴻海高薪轉戰調查局！台大電眼妹低調求學　爸「超硬背景」曝…同學全不知 https://www.ettoday.net/news/20191226/1610714.htm
2019/12/26 14:40 生活 2020年「新制懶人包」一次看！時薪150→158元、電動機車補助少了 https://www.ettoday.net/news/20191226/1610822.htm
2019/12/26 14:39 體育 聖誕大賽又爆冷！　英格雷31分領鵜鶘斬斷金塊7連勝 https://www.ettoday.net/news/20191226/1610910.htm
2019/12/26 14:39 消費 【廣編】樂天市場搶先公布2020消費新趨勢　STEM程式教育掀熱潮 https://www.ettoday.net/news/20191226/1610908.htm
2019/12/26 14:38 財經 200餘名台電工程師遭年末資遣　勞團要求「派遣歸零」 https://www.ettoday.net/news/20191226/1610906.htm
2019/12/26 14:34 財經 中租前11月獲利142億　每股盈餘破10元賺逾一個股本 https://www.ettoday.net/news/20191226/1610896.htm
2019/12/26 14:34 政治 告對手林宜瑾違反選罷法　

## 3. 根據範例，取出三天前下午三點到五點的新聞

In [7]:
import datetime

# 得到三天前的日期
today = datetime.date.today() 
three_days_ago = today - datetime.timedelta(3)
print(three_days_ago)

2019-12-23


In [8]:
# 三天前15:00的時間戳
dt = "2019/12/23 15:00"
timeArray = time.strptime(dt, "%Y/%m/%d %H:%M")
three_clock = time.mktime(timeArray)

# 三天前17:00的時間戳
dt = "2019/12/23 17:00"
timeArray = time.strptime(dt, "%Y/%m/%d %H:%M")
five_clock = time.mktime(timeArray)

In [9]:
# 選到三天前的日期，按下查詢按鈕
from selenium.webdriver.support.ui import Select

selectYear = Select(driver.find_element_by_id("selY"))
selectYear.select_by_value('2019')
selectMonth = Select(driver.find_element_by_id("selM"))
selectMonth.select_by_value('12')
selectDay = Select(driver.find_element_by_id('selD'))
selectDay.select_by_value('23')

driver.find_element_by_css_selector('#button').click()

In [10]:
# 每2秒執行一次滾動視窗的JS程式碼
for i in range(10):
    time.sleep(2)
    driver.execute_script('window.scrollTo(1,10000);')

In [16]:
# 滾完了後，獲取html原始碼

html = driver.page_source
soup = BeautifulSoup(html, 'html5lib')
for h3 in soup.find(class_='part_list_2').find_all('h3'):
    date_string = h3.find(class_='date').text
    # 如果時戳比15:00大 and 比17:00小
    if five_clock > str_to_timestamp(date_string) > three_clock :
        Date = h3.find(class_='date').text
        Title = h3.find(name='a').text
        Category = h3.find(name='em').text
        Link = h3.find(name='a')['href']
        print(Date, Category, Title, 'https://www.ettoday.net'+Link)

2019/12/23 16:58 財經 主計總處：11月失業率3.73%　就業人數增幅創10年新低　 https://www.ettoday.net/news/20191223/1608545.htm
2019/12/23 16:57 政治 快訊／韓國瑜台南升旗轉彎了！7點40分在高雄9點10分再到台南 https://www.ettoday.net/news/20191223/1608540.htm
2019/12/23 16:57 3C家電 三星新一代無線耳機Galaxy Buds將問世　電池續航時間更長！ https://www.ettoday.net/news/20191223/1608222.htm
2019/12/23 16:57 探索 愛運動飽受心臟無端電擊　心律不整患者可用皮下去顫器 https://www.ettoday.net/news/20191223/1606225.htm
2019/12/23 16:56 房產雲 民宿業者充當聖誕老公公　禮物竟是帶中途之家孩子「飛上天」 https://www.ettoday.net/news/20191223/1608532.htm
2019/12/23 16:54 體育 中職／統一獅提升訓練環境　整修室內練習場 https://www.ettoday.net/news/20191223/1608490.htm
2019/12/23 16:53 政治 沼田幹夫估蔡英文、韓國瑜差距5％內　賴清德：不可大意！ https://www.ettoday.net/news/20191223/1608544.htm
2019/12/23 16:52 社會 昔空軍中校變共諜！利誘現役軍官「消極不作為」：發揮關鍵作用　遭檢察官起訴 https://www.ettoday.net/news/20191223/1608122.htm
2019/12/23 16:51 財經 影／台股何時衝破歷史高點12682點？分析師：年前機會大 https://www.ettoday.net/news/20191223/1608431.htm
2019/12/23 16:51 寵物動物 上完沒？16歲老貓「臭臉走來」等廁所　炸毛背後原因好心疼 https://www.ettoday.net/news/20191223/1607874.htm

In [17]:
driver.close()